# QMbead code
to repeat 200 times for each case, see if any case will lead to wrong result.

In [1]:

import pennylane as qml

import math
import numpy as np
import matplotlib.pyplot as plt
import decimal
# from func_lib_VQF_QFT_exponent import my_QFT, my_inv_QFT, convert_num_integer_to_superposition_state, convert_num_bin_string_to_superposition_state
    

In [2]:
device_name = 'default.qubit'  # 'default.qubit' #
print("PennyLane version:", qml.__version__)

PennyLane version: 0.32.0


# functions used
## get_scale_coef,  plot_counts,  my_QFT,  my_inv_QFT
## convert_num_integer_to_superposition_state, 
## convert_num_bin_string_to_superposition_state

In [3]:

# Get the minimum 10**n such that decimal_val*10**n is an
#   integer number, where n is an integer
def get_scale_coef(decimal_val):
    n = 0
    while True:
        result = decimal_val * 10**n
        if result.is_integer():
            break
        n += 1
    return 10**n


def plot_counts(counts):
    # Extract keys and values
    keys = list(counts.keys())
    values = list(counts.values())

    # Create a bar plot
    plt.bar(keys, values)

    # Add labels and title
    plt.xlabel('Exponent')
    # plt.ylabel('Counts')
    # plt.title('Bar Plot of Counts')

    # Rotate x-axis labels for better readability
    plt.xticks(rotation=90)

        
def my_QFT(wires, swap_flag=True):
    for i in range(len(wires)):
        qml.Hadamard(wires=wires[i])
        for j in range(i+1, len(wires)):
            qml.CRZ(math.pi/2**(j-i),wires=[wires[i],wires[j]])
        qml.Barrier(wires)
        
    if swap_flag:
        n_wires = len(wires)
        first_half_wires = wires[: n_wires // 2]
        last_half_wires = wires[-(n_wires // 2) :]    
        for wire1, wire2 in zip(first_half_wires, reversed(last_half_wires)):
            qml.SWAP(wires=[wire1, wire2])
        
        
def my_inv_QFT(wires, swap_flag=True):
    ll = list(range(len(wires)))
    
    if swap_flag:
        n_wires = len(wires)
        first_half_wires = wires[: n_wires // 2]
        last_half_wires = wires[-(n_wires // 2) :]
        for wire1, wire2 in zip(reversed(first_half_wires), last_half_wires):
            qml.SWAP(wires=[wire1, wire2])
        
    for i in ll[::-1]:
        for j in range(i+1, len(wires)):
            qml.CRZ(-math.pi/2**(j-i),wires=[wires[i],wires[j]])
        qml.Hadamard(wires=wires[i])
        qml.Barrier(wires)
        
        
def convert_num_integer_to_superposition_state(num, print_flag=False):  #
    # Examples: 
    # 37 --> '100101' --> state [1, 0, 1, 0, 0, 1, 0, 0], it represents (|000> + |010> + |101>)/sqrt(3)    37 = 100101
    # 15 --> '1111' --> state [1, 1, 1, 1], it represents (|00> + |01> + |10> + |11>)/2                    15 = 1111
    # 16 --> '10000' --> state [0, 0, 0, 0, 1, 0, 0, 0], it represents |100>                               16 = 10000
    # integer_value = 37  # Replace with your desired integer value
    # result_list = convert_num_integer_to_superposition_state(integer_value)
    # print(result_list)
    # # num, binary form, position_list= 37  100101  [0, 2, 5]   37=2^0 + 2^2 + 2^5
    # # [1, 0, 1, 0, 0, 1, 0, 0], it represents (|000> + |010> + |101>)/sqrt(3)
    
    # integer_value = 15  # Replace with your desired integer value
    # result_list = convert_num_integer_to_superposition_state(integer_value)
    # print(result_list)
    # # num, binary form, position_list= 15  1111    [0, 1, 2, 3]   15=2^0 + 2^1 + 2^2 + 2^3
    # # [1, 1, 1, 1], it represents (|00> + |01> + |10> + |11>)/2
    
    # integer_value = 16  # Replace with your desired integer value
    # result_list = convert_num_integer_to_superposition_state(integer_value)
    # print(result_list)
    # # num, binary form, position_list= 16  10000   [4]    16=2^4
    # # [0, 0, 0, 0, 1, 0, 0, 0], it represents |100>
    s1 = bin(num)[2:]  # Convert integer to binary string without '0b'
    state, nq = convert_num_bin_string_to_superposition_state(s1, print_flag=print_flag)
    return state, nq


def convert_num_bin_string_to_superposition_state(s1, print_flag=True):
    # Examples:
    # '100101' --> state [1, 0, 1, 0, 0, 1, 0, 0], it represents (|000> + |010> + |101>)/sqrt(3)    37 = 100101
    # '1111' --> state [1, 1, 1, 1], it represents (|00> + |01> + |10> + |11>)/2                    15 = 1111
    # '10000' --> state [0, 0, 0, 0, 1, 0, 0, 0], it represents |100>                               16 = 10000
    # see Examples given in convert_num_integer_to_superposition_state(num) above for more information
    # + convert an integer into a binary string s1, 
    # + the right most bit's position is 0 and leftmost bit's position is most significant.
    # + for each bit in s1, if its value is 1,   add this bit's position into a list: position_list
    # + find the maximum value in the position_list: max_val
    # + find the minimum n such that 2**n >= max_val
    # + create an all-0 list: state
    # + for the state, change the 0 to 1 for each position in position_list
    # + normalize the state
    # return the list and nq, where nq is the number of qubits
    
    # the position_list is a list of positions of '1' counted from rightmost, start from 0
    position_list = [pos for pos, bit in enumerate(s1[::-1]) if bit == '1']
    if print_flag: print('num, binary form, position_list=', int(s1), s1, position_list)
    max_val = max(position_list, default=-1) + 1  # Get the maximum value in position_list
    
    n = 0
    while 2 ** n < max_val:
        n += 1
        
    state = [0] * (2**n)
    for pos in position_list:
        state[pos] = 1
        
    norm = np.linalg.norm(state)  # Calculate the norm of the original list
    normalized_state = [element / norm for element in state]
    
    nq = n # the number of qubits
    return normalized_state, nq



# function: get_product_solution_from_counts()

In [4]:

def get_product_solution_from_counts(counts, print_flag = True):
    
    # Find the minimum value in the dictionary
    min_value = min(counts.values())

    # Divide all values by the minimum value and round to the nearest integer
    updated_counts = {key: round(value / min_value) for key, value in counts.items()}
    # Print the updated dictionary
    if print_flag: 
        print(f'updated_counts={updated_counts}')
    
        plt.figure(figsize=(9, 5))
        # plot the counts and updated_counts side by side
        plt.subplot(1,2,1)
        plot_counts(counts)    
        plt.ylabel('Counts')
        
        plt.subplot(1,2,2)
        plot_counts(updated_counts)
        plt.ylabel('Coefficient')
        
        # Show the plot
        plt.show()
    
    
    # calculate the product value u*v = sum_k (coef*2^k)
    sum0 = 0
    for key, value in updated_counts.items():
        sum0 = sum0 + value * 2**(int(key, 2))
        # print(f'key={key}, int_key={int(key, 2)}, value={value}, sum0={sum0}')
        
    return sum0
    

# function: addition_QFT_Draper()  &  addition_QFT_Draper2()

In [5]:

def addition_QFT_Draper(wires_b, wires_a): # this function's output in wires_a can have overflow, solved by using addition_QFT_Draper2()
    qml.Barrier(wires_b+wires_a)
    my_QFT(wires = wires_a, swap_flag = False)
    
    lenb = len(wires_b)
    for i in range(lenb):
        for j in range(lenb-i):
            # print('i,j=', i, j)
            qml.CRZ(math.pi/2**(j), wires=[wires_b[i+j],wires_a[i]])
    qml.Barrier(wires_b+wires_a)
    
    my_inv_QFT(wires=wires_a, swap_flag = False)
    
    
def addition_QFT_Draper2(wires_b, wires_a): # address the overflow problem
    # lena can be larger than or equal to lenb
    qml.Barrier(wires_b+wires_a)
    my_QFT(wires = wires_a, swap_flag = False)
    qml.Barrier(wires_b+wires_a)
    
    lenb = len(wires_b)
    lena = len(wires_a) # lena can be larger than or equal to lenb
    diff = lena-lenb
    for i in range(lena):
        start = max(diff - i, 0)        
        for j in range(start,lena-i):
            # print('i,j=', i, j)
            qml.CRZ(math.pi/2**(j), wires=[wires_b[i+j-diff],wires_a[i]])
    qml.Barrier(wires_b+wires_a)
    
    my_inv_QFT(wires=wires_a, swap_flag = False)
    
    

# function: get_wires_states

In [6]:

def get_wires_states(u, v, print_flag=True):
    state_u, n_u = convert_num_integer_to_superposition_state(u, print_flag=print_flag)
    state_v, n_v = convert_num_integer_to_superposition_state(v, print_flag=print_flag)
    if print_flag:
        print(f'state_u={state_u}, n_u={n_u}')
        print(f'state_v={state_v}, n_v={n_v}')
    
    # state_b should use less number of qubits than state_a
    n_b = min(n_u, n_v)
    n_a = max(n_u, n_v)
    if n_u <= n_v:  # n_u is smaller
        state_b = state_u
        state_a = state_v
    else:           # n_v is smaller
        state_b = state_v 
        state_a = state_u
    
    # first wires_b  (n_b qubits)
    # followed by wires_carry for a carry qubit  (1 qubit)
    # followed by wires_a  (n_a qubits)
    wires_b = list(range(n_b))
    wires_carry = [n_b]
    wires_a = [i+n_b+1 for i in range(n_a)]
    
    return wires_b, wires_carry, wires_a, state_b, state_a



# Class: QMbead
## func: run_main_circ_return_counts
## func: run_main_circ_return_state   # to delete
## func: run_main_circ_return_expval # to delete
## func: main_circ
## func: run

In [7]:

class QMbead:
    def __init__(self, u, v, mode = 1,  
                 dev_counts_num = 100000, 
                 print_flag=False):
        self.u = u
        self.v = v
        self.measure_type = 'counts'  # measure_type
        # self.wire_anx = wire_anx
        # self.wires_1 = wires_1         
        self.print_flag = print_flag
        self.mode= mode
        
        self.wires_b, self.wires_carry, self.wires_a, \
            self.state_b, self.state_a = get_wires_states(u, v, print_flag=print_flag)  
        all_wires = self.wires_b + self.wires_carry + self.wires_a
        
        # for more qubits, use larger counts
        nw = len(all_wires)
        if print_flag: 
            print(f'total number of qubits used: {nw},  ', end='')
            print(f'n_b={len(self.wires_b)},   ', end='')
            print(f'n_a={len(self.wires_a)}')
        # if nw < 10:
        #     dev_counts_num = 100000
        # elif nw < 12:
        #     dev_counts_num = 1000000    
        # elif nw < 14:
        #     dev_counts_num = 10000000
        self.dev_counts_num = dev_counts_num
        
        if self.measure_type in ['counts', 'sample']:
            self.dev = qml.device(device_name, all_wires, shots=self.dev_counts_num)
        elif self.measure_type in ['sample_all']:
            self.dev = qml.device(device_name, all_wires, shots=2000)
        elif self.measure_type in ['state', 'expval']:
            self.dev = qml.device(device_name, all_wires)
            
        self.dev_counts = qml.device(device_name, all_wires, shots=self.dev_counts_num)
        self.dev_expval = qml.device(device_name, all_wires)
        
    def run_main_circ_return_counts(self):
        @qml.qnode(self.dev_counts)
        def _run_main_circ():
            self.main_circ()
            # lenb = len(self.wires_1)//2
            # wires_count = self.wires_1[lenb:]
            return qml.counts(wires = self.wires_carry + self.wires_a)
        return _run_main_circ()
    
    
    def run_main_circ_return_state(self):
        @qml.qnode(self.dev_counts)
        def _run_main_circ():
            self.main_circ()
            return qml.state()
        return _run_main_circ() 
    
    
    def run_main_circ_return_expval(self):
        @qml.qnode(self.dev_expval)
        def _run_main_circ_return_expval():
            self.main_circ()
            return qml.expval(qml.PauliZ([0])) # measure qubit 0
        if self.print_flag: print(qml.draw(_run_main_circ_return_expval)())
        # print('')
        # if self.print_flag: print(qml.draw(_run_main_circ_return_expval, 
        #                                    expansion_strategy="device")())
        return _run_main_circ_return_expval()
    
    
    def main_circ(self):
        # if self.mode == 1:  # not successful. So I go back to the using the circuit given in https://pennylane.ai/qml/demos/tutorial_qft_arithmetics
        qml.MottonenStatePreparation(self.state_b, wires=self.wires_b)
        qml.MottonenStatePreparation(self.state_a, wires=self.wires_a)    
        
        addition_QFT_Draper2(self.wires_b, self.wires_carry+self.wires_a)
        
        # elif self.mode == 2: # test addition_QFT_Draper(), come from Draper2000 paper, 
            
         
    def run(self, print_flag=True):
        Z_exp = self.run_main_circ_return_expval()
        # print('Z_exp=', Z_exp)
                
        # state = self.run_main_circ_return_state()
        # print('state=', state)
        
        counts = self.run_main_circ_return_counts()
        if print_flag: print('counts=', counts)
        
        product = get_product_solution_from_counts(counts, print_flag=print_flag)
        if print_flag: 
            print(f'product of u={self.u} and v={self.v}  is  {product}, ', end='')
            print(f'its binary form is {bin(product)} whose length is {len(bin(product))-2}')
        
        uv = self.u*self.v # for verification
        if uv == product:
            if print_flag: print(f'The product calculated by QMbead is correct')
            product_correct = True
        else:
            print(f'The product calculated by QMbead is wrong, correct value should be {uv}')
            product_correct = False
            if print_flag: raise ValueError('wrong result by QMbead')
        if print_flag: print('')
        return product, product_correct
        

# Results for integer multiplication

In [8]:
u = 3; v = 5
correct_count = 0
repeat_num = 200
for repeat in range(repeat_num):
    circ = QMbead(u, v, dev_counts_num=int(1e3), print_flag=False);  
    product, product_correct=circ.run(print_flag = False)
    if product_correct:
        correct_count += 1
print('correct_count=', correct_count)

correct_count= 200


In [9]:
u = 7; v = 11
correct_count = 0
repeat_num = 200
for repeat in range(repeat_num):
    circ = QMbead(u, v, dev_counts_num=int(1e4), print_flag=False);  
    product, product_correct=circ.run(print_flag = False)
    if product_correct:
        correct_count += 1
print('correct_count=', correct_count)

correct_count= 200


In [10]:
u = 33; v = 100
correct_count = 0
repeat_num = 200
for repeat in range(repeat_num):
    circ = QMbead(u, v, dev_counts_num=int(1e5), print_flag=False);  
    product, product_correct=circ.run(print_flag = False)
    if product_correct:
        correct_count += 1
print('correct_count=', correct_count)

correct_count= 200


In [11]:
u = 2345; v = 5678
correct_count = 0
repeat_num = 200
for repeat in range(repeat_num):
    circ = QMbead(u, v, dev_counts_num=int(1e5), print_flag=False);  
    product, product_correct=circ.run(print_flag = False)
    if product_correct:
        correct_count += 1
print('correct_count=', correct_count)

correct_count= 200


In [12]:
u = 234501; v = 567801
correct_count = 0
repeat_num = 200
for repeat in range(repeat_num):
    circ = QMbead(u, v, dev_counts_num=int(1e5), print_flag=False);  
    product, product_correct=circ.run(print_flag = False)
    if product_correct:
        correct_count += 1
print('correct_count=', correct_count)

The product calculated by QMbead is wrong, correct value should be 133149902301
The product calculated by QMbead is wrong, correct value should be 133149902301
The product calculated by QMbead is wrong, correct value should be 133149902301
correct_count= 197


In [13]:
u = 23450101; v = 56780101
correct_count = 0
repeat_num = 200
for repeat in range(repeat_num):
    circ = QMbead(u, v, dev_counts_num=int(1e6), print_flag=False);   
    product, product_correct=circ.run(print_flag = False)
    if product_correct:
        correct_count += 1
print('correct_count=', correct_count)

correct_count= 200


In [14]:
u = 2345010101; v = 5678010101
correct_count = 0
repeat_num = 200
for repeat in range(repeat_num):
    circ = QMbead(u, v, dev_counts_num=int(1e6), print_flag=False);   
    product, product_correct=circ.run(print_flag = False)
    if product_correct:
        correct_count += 1
print('correct_count=', correct_count)

The product calculated by QMbead is wrong, correct value should be 13314991040425030201
The product calculated by QMbead is wrong, correct value should be 13314991040425030201
The product calculated by QMbead is wrong, correct value should be 13314991040425030201
The product calculated by QMbead is wrong, correct value should be 13314991040425030201
correct_count= 196


In [15]:
u = 8978923748987; v = 8984957438475849
correct_count = 0
repeat_num = 200
for repeat in range(repeat_num):
    circ = QMbead(u, v, dev_counts_num=int(5e6), print_flag=False);   
    product, product_correct=circ.run(print_flag = False)
    if product_correct:
        correct_count += 1
print('correct_count=', correct_count)

The product calculated by QMbead is wrong, correct value should be 80675247727968202502337714963
The product calculated by QMbead is wrong, correct value should be 80675247727968202502337714963
The product calculated by QMbead is wrong, correct value should be 80675247727968202502337714963
The product calculated by QMbead is wrong, correct value should be 80675247727968202502337714963
The product calculated by QMbead is wrong, correct value should be 80675247727968202502337714963
The product calculated by QMbead is wrong, correct value should be 80675247727968202502337714963
The product calculated by QMbead is wrong, correct value should be 80675247727968202502337714963
The product calculated by QMbead is wrong, correct value should be 80675247727968202502337714963
The product calculated by QMbead is wrong, correct value should be 80675247727968202502337714963
The product calculated by QMbead is wrong, correct value should be 80675247727968202502337714963
The product calculated by QMbe

# Results for decimal multiplication

In [20]:
u = 0.567; v = 0.0004
correct_count = 0
repeat_num = 200
for repeat in range(repeat_num):
    coef_u = get_scale_coef(u);    coef_v = get_scale_coef(v)
    circ = QMbead(int(u*coef_u), int(v*coef_v), dev_counts_num=int(1e5), print_flag=False);  
    product, product_correct=circ.run(print_flag = False)
    if product_correct:
        correct_count += 1    
    # product = circ.run()/coef_u/coef_v
    # print(f'the product of the decimal values is  {product:.10f}')
    
print('correct_count=', correct_count)

correct_count= 200


In [21]:
u = 2.5; v = 1.75
correct_count = 0
repeat_num = 200
for repeat in range(repeat_num):
    coef_u = get_scale_coef(u);    coef_v = get_scale_coef(v)
    circ = QMbead(int(u*coef_u), int(v*coef_v), dev_counts_num=int(1e5), print_flag=False);  
    product, product_correct=circ.run(print_flag = False)
    if product_correct:
        correct_count += 1    
    # product = circ.run()/coef_u/coef_v
    # print(f'the product of the decimal values is  {product:.5f}')
print('correct_count=', correct_count)

correct_count= 200


In [9]:

# Set the precision to a high value
decimal.getcontext().prec = 100
u = 136872.345502; v = 2343651.74543455
correct_count = 0
repeat_num = 200
for repeat in range(repeat_num):
    coef_u = get_scale_coef(u);    coef_v = get_scale_coef(v)
    circ = QMbead(int(u*coef_u), int(v*coef_v), dev_counts_num=int(5e6), print_flag=False);  
    product, product_correct=circ.run(print_flag = False)
    if product_correct:
        correct_count += 1 
    # product_decimal = decimal.Decimal(str(product))/decimal.Decimal(str(coef_u*coef_v))
    # print(f'the product of the decimal values is  {product_decimal:.15f}')    
print('correct_count=', correct_count)

The product calculated by QMbead is wrong, correct value should be 320781111437483078727894100
The product calculated by QMbead is wrong, correct value should be 320781111437483078727894100
The product calculated by QMbead is wrong, correct value should be 320781111437483078727894100
correct_count= 197
